In [16]:
import pandas as pd
import sqlite3
import numpy as np
import matplotlib.pyplot as plt
from itertools import product
from scipy.spatial.distance import cdist, pdist
import seaborn as sns

def idxwhere(x):
    return x[x].index

## Smith 2019 Data

In [17]:
con2019 = sqlite3.connect('../longev/res/C2013.results.db')

In [18]:
unique_to_otu2019 = pd.read_sql(
    """
    SELECT taxon_id, taxon_id_b
    FROM taxonomy
    WHERE taxon_level = 'unique'
      AND taxon_level_b = 'otu-0.03'
    """,
    index_col=['taxon_id'],
    con=con2019,
).squeeze()

In [19]:
otu_taxonomy2019 = pd.read_sql(
    """
    SELECT taxon_id, taxon_level_b, taxon_id_b FROM taxonomy
    WHERE taxon_level = 'otu-0.03'
    """,
    index_col=['taxon_id', 'taxon_level_b'],
    con=con2019,
).squeeze().unstack()[['phylum', 'class', 'order', 'family', 'genus']]

In [20]:
count2019 = pd.read_sql(
    """
    SELECT extraction_id, taxon_id, SUM(tally) AS tally
    FROM rrs_library_taxon_count
    JOIN rrs_library USING (rrs_library_id)
    GROUP BY extraction_id, taxon_id
    """,
    index_col=['extraction_id', 'taxon_id'],
    con=con2019,
).squeeze().unstack(fill_value=0).groupby(unique_to_otu2019, axis='columns').sum()

In [21]:
otu_taxonomy2019[otu_taxonomy2019.family == 'Muribaculaceae'].head(10)

taxon_level_b,phylum,class,order,family,genus
taxon_id,,,,,
Otu0001,Bacteroidetes,Bacteroidia,Bacteroidales,Muribaculaceae,Muribaculaceae_ge
Otu0004,Bacteroidetes,Bacteroidia,Bacteroidales,Muribaculaceae,Muribaculaceae_ge
Otu0005,Bacteroidetes,Bacteroidia,Bacteroidales,Muribaculaceae,Muribaculaceae_ge
Otu0006,Bacteroidetes,Bacteroidia,Bacteroidales,Muribaculaceae,Muribaculaceae_ge
Otu0008,Bacteroidetes,Bacteroidia,Bacteroidales,Muribaculaceae,Muribaculaceae_ge
Otu0011,Bacteroidetes,Bacteroidia,Bacteroidales,Muribaculaceae,Muribaculaceae_ge
Otu0030,Bacteroidetes,Bacteroidia,Bacteroidales,Muribaculaceae,Muribaculaceae_ge
Otu0039,Bacteroidetes,Bacteroidia,Bacteroidales,Muribaculaceae,Muribaculaceae_ge
Otu0060,Bacteroidetes,Bacteroidia,Bacteroidales,Muribaculaceae,Muribaculaceae_ge


## Smith2020 Data

In [22]:
con2020 = sqlite3.connect('data/core.muri2.2.denorm.db')

In [23]:
count2020 = (pd.read_sql(
        """
        SELECT extraction_id, otu_id, SUM(tally) AS tally
        FROM rrs_taxon_count
        GROUP BY extraction_id, otu_id
        """,
        con=con2020, index_col=['extraction_id', 'otu_id'])
    .squeeze().unstack().fillna(0))

In [24]:
otu_taxonomy2020 = pd.read_sql(
    """
    SELECT DISTINCT otu_id, domain_, phylum_, class_, order_, family_, genus_ FROM rrs_taxonomy
    """,
    index_col='otu_id',
    con=con2020,
)

In [25]:
otu_taxonomy2020[otu_taxonomy2020.family_ == 'Muribaculaceae'].head(10)

,domain_,phylum_,class_,order_,family_,genus_
otu_id,,,,,,
Otu0001,Bacteria,Bacteroidetes,Bacteroidia,Bacteroidales,Muribaculaceae,Muribaculaceae_ge
Otu0004,Bacteria,Bacteroidetes,Bacteroidia,Bacteroidales,Muribaculaceae,Muribaculaceae_ge
Otu0005,Bacteria,Bacteroidetes,Bacteroidia,Bacteroidales,Muribaculaceae,Muribaculaceae_ge
Otu0007,Bacteria,Bacteroidetes,Bacteroidia,Bacteroidales,Muribaculaceae,Muribaculaceae_ge
Otu0009,Bacteria,Bacteroidetes,Bacteroidia,Bacteroidales,Muribaculaceae,Muribaculaceae_ge
Otu0010,Bacteria,Bacteroidetes,Bacteroidia,Bacteroidales,Muribaculaceae,Muribaculaceae_ge
Otu0013,Bacteria,Bacteroidetes,Bacteroidia,Bacteroidales,Muribaculaceae,Muribaculaceae_ge
Otu0014,Bacteria,Bacteroidetes,Bacteroidia,Bacteroidales,Muribaculaceae,Muribaculaceae_ge
Otu0017,Bacteria,Bacteroidetes,Bacteroidia,Bacteroidales,Muribaculaceae,Muribaculaceae_ge


## Matching

In [26]:
count2019

taxon_id_b,Otu0001,Otu0002,Otu0003,Otu0004,Otu0005,Otu0006,Otu0007,Otu0008,Otu0009,Otu0010,...,Otu1774,Otu1843,Otu1874,Otu1880,Otu1922,Otu1964,Otu2075,Otu2079,Otu2096,Otu2392
extraction_id,,,,,,,,,,,,,,,,,,,,,
EXT-0029,1539,1137,1026,0,637,616,5044,155,156,13,...,0,0,0,0,0,0,0,0,0,0
EXT-0031,1594,608,129,4196,0,6,108,0,3,451,...,0,0,0,0,0,0,0,0,0,0
EXT-0032,10226,590,691,3,605,2,1,781,128,0,...,0,0,0,0,0,0,0,0,0,0
EXT-0037,543,186,621,0,916,2,3747,1218,258,117,...,0,0,0,0,0,0,0,0,0,0
EXT-0040,5564,27,196,0,1718,3,2,1046,201,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
EXT-0428,980,261,1,1,275,0,2370,291,74,17,...,0,0,0,0,0,0,0,0,0,0
EXT-0429,9506,86,0,0,4,0,3,1892,593,24,...,0,0,0,0,0,0,0,0,0,0
EXT-0430,2090,1902,1,0,2392,0,1792,630,345,62,...,0,0,0,0,0,0,0,0,0,0


In [27]:
count2020.loc[count2019.index]

otu_id,Otu0001,Otu0002,Otu0003,Otu0004,Otu0005,Otu0006,Otu0007,Otu0008,Otu0009,Otu0010,...,Otu5277,Otu5278,Otu5279,Otu5280,Otu5281,Otu5282,Otu5283,Otu5284,Otu5285,Otu5286
extraction_id,,,,,,,,,,,,,,,,,,,,,
EXT-0029,1571.0,1159.0,1046.0,626.0,680.0,5155.0,0.0,1.0,157.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
EXT-0031,1658.0,625.0,133.0,6.0,0.0,112.0,4277.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
EXT-0032,10441.0,599.0,720.0,2.0,658.0,1.0,3.0,1.0,800.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
EXT-0037,554.0,187.0,634.0,2.0,977.0,3894.0,0.0,1.0,1259.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
EXT-0040,5710.0,28.0,201.0,3.0,1834.0,2.0,0.0,2.0,1088.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
EXT-0428,1022.0,268.0,1.0,0.0,292.0,2450.0,1.0,0.0,297.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
EXT-0429,9709.0,90.0,0.0,0.0,6.0,3.0,0.0,0.0,1936.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
EXT-0430,2157.0,1936.0,1.0,0.0,2567.0,1846.0,0.0,0.0,653.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [28]:
muri_otus2019 = idxwhere(otu_taxonomy2019.loc[count2019.columns].family == 'Muribaculaceae')
muri_otus2020 = idxwhere((otu_taxonomy2020.loc[count2020.columns].family_ == 'Muribaculaceae')
                         & count2020.loc[count2019.index].sum() > 0)

# fig, axs = plt.subplots(nrows=8, ncols=8, figsize=(15, 15))

# for otu2019, row in zip(muri_otus2019, axs):
#     for otu2020, ax in zip(muri_otus2020, row):
#         ax.scatter(count2019[otu2019], count2020.loc[count2019.index, otu2020])

In [29]:
dmat_corr = pd.DataFrame(
    cdist(
        count2019.loc[:, muri_otus2019].T,
        count2020.loc[count2019.index, muri_otus2020].T,
        metric='correlation',
    ),
    index=muri_otus2019,
    columns=muri_otus2020,
).rename_axis(index='otus2019', columns='otus2020')

dmat_cb = pd.DataFrame(
    cdist(
        count2019.loc[:, muri_otus2019].T,
        count2020.loc[count2019.index, muri_otus2020].T,
        metric='cityblock',
    ),
    index=muri_otus2019,
    columns=muri_otus2020,
).rename_axis(index='otus2019', columns='otus2020')

In [30]:
best_hit = pd.DataFrame({
    'corr_hit': dmat_corr.idxmin(),
    'corr': dmat_corr.min(),
    'cb_hit': dmat_cb.idxmin(),
    'cb': dmat_cb.min(),
    'total2020': count2020.loc[count2019.index].sum(),
})#.dropna()

#total2020 = 
best_hit.join(count2019.sum().rename('total2019'), on='corr_hit').loc[muri_otus2020].head(20)

,corr_hit,corr,cb_hit,cb,total2020,total2019
otu_id,,,,,,
Otu0001,Otu0001,1.683127e-05,Otu0001,15697.0,697276.0,681583.0
Otu0004,Otu0006,3.515030e-05,Otu0006,3961.0,127860.0,123903.0
Otu0005,Otu0005,3.306074e-05,Otu0005,7930.0,128518.0,120590.0
Otu0007,Otu0004,7.163800e-06,Otu0004,3690.0,181066.0,177376.0
Otu0009,Otu0008,2.440454e-05,Otu0008,2151.0,81202.0,79051.0
Otu0010,Otu0473,1.447436e-01,Otu0473,13.0,43.0,30.0
Otu0013,Otu0011,2.620290e-05,Otu0011,2203.0,41423.0,39220.0
Otu0014,Otu0060,7.856934e-04,Otu0060,1035.0,9182.0,8147.0
Otu0017,Otu0030,9.554748e-05,Otu0030,1064.0,22637.0,21573.0
